In [27]:
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
from pprint import pprint
import json
import requests
import numpy as np
import pandas as pd

from src.params import *
from src.nws import NWSClient
from src.nws import (
    fahrenheit_to_celsius,
    celsius_to_fahrenheit,
    load_clis,
    load_one_minutes,
    end_of_day,
    start_of_day,
    in_date_range
)

In [28]:
station = StationID.MDW

clis = load_clis(station)

pprint([cli.without_raw_text() for cli in clis])

[CLI(issuance_time=datetime.datetime(2024, 11, 6, 16, 51, tzinfo=<DstTzInfo 'America/Chicago' CST-1 day, 18:00:00 STD>),
     issuing_office='LOT',
     summary_date=datetime.date(2024, 11, 6),
     raw_text='...',
     max_temp=61,
     max_temp_time=datetime.datetime(2024, 11, 6, 0, 50, tzinfo=<DstTzInfo 'America/Chicago' CST-1 day, 18:00:00 STD>),
     min_temp=52,
     min_temp_time=datetime.datetime(2024, 11, 6, 15, 2, tzinfo=<DstTzInfo 'America/Chicago' CST-1 day, 18:00:00 STD>),
     avg_temp=57,
     valid_time=datetime.datetime(2024, 11, 6, 16, 0, tzinfo=<DstTzInfo 'America/Chicago' CST-1 day, 18:00:00 STD>)),
 CLI(issuance_time=datetime.datetime(2024, 11, 7, 0, 41, tzinfo=<DstTzInfo 'America/Chicago' CST-1 day, 18:00:00 STD>),
     issuing_office='LOT',
     summary_date=datetime.date(2024, 11, 6),
     raw_text='...',
     max_temp=61,
     max_temp_time=datetime.datetime(2024, 11, 6, 0, 50, tzinfo=<DstTzInfo 'America/Chicago' CST-1 day, 18:00:00 STD>),
     min_temp=51,
   

In [29]:
# Sort clis by date
clis = sorted(clis, key=lambda cli: (cli.summary_date, cli.issuance_time))

In [30]:
from dataclasses import asdict

# Make a dataframe that includes all the issuance times of all the clis
df = pd.DataFrame([asdict(cli.without_raw_text()) for cli in clis])
df.drop(columns=['raw_text'], inplace=True)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype                          
---  ------          --------------  -----                          
 0   issuance_time   55 non-null     datetime64[ns, America/Chicago]
 1   issuing_office  55 non-null     object                         
 2   summary_date    55 non-null     object                         
 3   max_temp        55 non-null     int64                          
 4   max_temp_time   54 non-null     datetime64[ns, America/Chicago]
 5   min_temp        55 non-null     int64                          
 6   min_temp_time   54 non-null     datetime64[ns, America/Chicago]
 7   avg_temp        55 non-null     int64                          
 8   valid_time      27 non-null     datetime64[ns, America/Chicago]
dtypes: datetime64[ns, America/Chicago](4), int64(3), object(2)
memory usage: 4.0+ KB


,issuance_time,issuing_office,summary_date,max_temp,max_temp_time,min_temp,min_temp_time,avg_temp,valid_time
0,2024-10-22 01:33:00-05:00,LOT,2024-10-21,82,2024-10-21 16:06:00-05:00,53,2024-10-21 04:07:00-05:00,68,NaT
1,2024-10-22 16:36:00-05:00,LOT,2024-10-22,74,2024-10-22 14:59:00-05:00,55,2024-10-22 06:18:00-05:00,65,2024-10-22 16:00:00-05:00
2,2024-10-23 01:34:00-05:00,LOT,2024-10-22,75,2024-10-22 15:45:00-05:00,55,2024-10-22 06:18:00-05:00,65,NaT
3,2024-10-23 16:36:00-05:00,LOT,2024-10-23,65,2024-10-23 14:47:00-05:00,58,2024-10-23 06:37:00-05:00,62,2024-10-23 16:00:00-05:00
4,2024-10-24 01:37:00-05:00,LOT,2024-10-23,65,NaT,46,NaT,56,NaT


In [31]:
# Plot the ranges of the CLI timeframes
import plotly.express as px

issue_df = df[["issuance_time"]].copy()
issue_df["issuance_time"] = issue_df["issuance_time"].dt.time
# Filter on those that are between 4-5pm
hours = range(16, 18)
issue_df = issue_df[issue_df["issuance_time"].apply(lambda t: t.hour in hours)]
issue_df["issuance_time"] = issue_df["issuance_time"].apply(lambda t: ((t.hour - min(hours)) * 60 + t.minute))

binsize = 1 # minute
fig = px.histogram(issue_df, x="issuance_time", nbins=180//binsize)
fig.show()


In [ ]:
# MIA Release times
# 16:21 to 16:40 Eastern
# NYC Release times
# 16:32 to 16:52 Eastern
# AUS Release times
# 16:22 - 17:59 Central
# MDW Release times
# 16:32 to 16:51 Central
